In [ ]:
from google.cloud.bigquery.client import Client
import pandas as pd
from sklearn.model_selection import train_test_split

from src.config import ALL_COLUMNS, LABEL_COLUMN

In [ ]:
proyect_id = "peya-data-analyt-factory-stg"
client = Client(project=proyect_id)

In [ ]:
query_path = "../data/queries/flood_it_dataset.sql"
with open(query_path, "r") as file:
    query = "".join(file.readlines())

full_dataset = client.query(query).to_dataframe(create_bqstorage_client=False)

In [ ]:
output_path = "../data/users_raw.csv"
full_dataset.to_csv(output_path, index=False)

In [ ]:
selector_train_test = (full_dataset["is_enable"] == 1) & (full_dataset["bounced"] == 0)
selectot_not_yet = (full_dataset["is_enable"] == 0) & (full_dataset["bounced"] == 0)

train, test = train_test_split(
    full_dataset.loc[selector_train_test],
    test_size=0.1,
    random_state=42,
    shuffle=True,
    stratify=full_dataset.loc[selector_train_test, LABEL_COLUMN],
)

train_path = "../data/users_train.csv"
test_path = "../data/users_test.csv"
not_yet_path = "../data/users_not_yet.csv"
train.to_csv(train_path, index=False)
test.to_csv(test_path, index=False)
full_dataset.loc[selectot_not_yet, :].to_csv(not_yet_path, index=False)